# 0. Introduction
1. We write `DpseTildeRPairV1` to construct the $\widetilde{R}$ in `DeepPot-SE` 
    - Note: For pair elements

In [1]:
import numpy as np
from timeit import default_timer as timer
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.pwmat.output.movement import Movement
from matersdk.io.publicLayer.neigh import (
                    StructureNeighborsDescriptor,
                    StructureNeighborsUtils
)
from matersdk.feature.deepmd.se_pair import DpseTildeRPairDescriptor

import warnings

warnings.filterwarnings('ignore')

# 1. Demo for `DpseTildeRPairV1`
Calculate the `Environment matrix` (Rij) for Li-Li pair

## 1.1. Calculate the `Environment matrix` (Rij) for Li-Li pair

In [6]:
movement_path = "/data/home/liuhanyu/hyliu/code/mlff/test/demo2/PWdata/data1/MOVEMENT"
structure = Movement(movement_path=movement_path).get_frame_structure(idx_frame=343)

scaling_matrix = [3, 3, 3]
reformat_mark = True
coords_are_cartesian = True

center_atomic_number = 3    # Li
nbr_atomic_numbers = [3, 14]      # Li, Si
max_num_nbrs = [100, 100]
rcut = 5
rcut_smooth = 0.5



struct_nbr = StructureNeighborsDescriptor.create(
                'v1',
                structure=structure,
                rcut=rcut,
                scaling_matrix=scaling_matrix,
                reformat_mark=reformat_mark,
                coords_are_cartesian=coords_are_cartesian)
dpse_tildeR_pair = DpseTildeRPairDescriptor.create(
                'v1',
                structure_neighbors=struct_nbr,
                center_atomic_number=center_atomic_number,
                nbr_atomic_number=nbr_atomic_numbers[0],    # Li
                rcut=rcut,
                rcut_smooth=rcut_smooth)


print()
print("Step 1. Print the attributions of DeepmdSeR:")
print("\t1. deepmd_se_r.dp_feature_pair_an.shape = ", dpse_tildeR_pair.dp_feature_pair_an.shape)
print("\t2. deepmd_se_r.dp_feature_pair_d.shape = ", dpse_tildeR_pair.dp_feature_pair_d.shape)
print("\t3. deepmd_se_r.dp_feature_pair_rc.shape = ", dpse_tildeR_pair.dp_feature_pair_rc.shape)


### Step 2. Get smooth edition s_{ij}
print()
print("Step 2. Get segmented form of s in Deepmd:")
print("\t1. s.shape = ", dpse_tildeR_pair._get_s(rcut=rcut, rcut_smooth=rcut_smooth).shape)


### Step 3.
print()
print("Step 3.")
print("\t1. deepmd_se_r.dp_feature_pair_tildeR.shape = ", dpse_tildeR_pair.dp_feature_pair_tildeR.shape)


Step 1. Print the attributions of DeepmdSeR:
	1. deepmd_se_r.dp_feature_pair_an.shape =  (48, 20)
	2. deepmd_se_r.dp_feature_pair_d.shape =  (48, 20)
	3. deepmd_se_r.dp_feature_pair_rc.shape =  (48, 20, 3)

Step 2. Get segmented form of s in Deepmd:
	1. s.shape =  (48, 20)

Step 3.
	1. deepmd_se_r.dp_feature_pair_tildeR.shape =  (48, 20, 4)


## 1.2. Zero-padding

In [7]:
print("1. Before zero-padding, Rij's shape = ", end='\t')
print(dpse_tildeR_pair.dp_feature_pair_tildeR.shape)
print("2. After zero-padding, Rij's shape = ", end='\t')
print(dpse_tildeR_pair.get_tildeR(max_num_nbrs=max_num_nbrs[0]).shape)

1. Before zero-padding, Rij's shape = 	(48, 20, 4)
2. After zero-padding, Rij's shape = 	(48, 100, 4)


# 1.3. Derivative of $\tilde{R}^i_j$ with respect to x, y, z

In [8]:
print("2. After zero-padding, the derivative of Rij's shape = ", end='\t')
print(dpse_tildeR_pair.calc_derivative(max_num_nbrs=max_num_nbrs[0]).shape)

2. After zero-padding, the derivative of Rij's shape = 	(48, 100, 4, 3)


# 2. For `PWmatMLFF`, get `neigh_list`
The `343st` is corresponding to `image_005`

In [9]:
neigh_list = StructureNeighborsUtils.get_nbrs_indices(
                    struct_nbr=struct_nbr,
                    center_atomic_number=center_atomic_number,
                    nbr_atomic_number=nbr_atomic_numbers[0])

In [11]:
np.sort(neigh_list[0])

array([ 0,  2,  2,  3,  3,  4,  4,  9, 10, 12, 13, 14, 15, 37, 38, 40, 41,
       42, 43, 45])

## 2.1. Compare with Fortran code
Compare with the forth column of `PWdata/dRneigh.dat`.

In [31]:
print(np.sort(neigh_list[0]))

[ 2  2  3  3  4  4  9 10 12 13 14 15 37 38 40 41 42 43 45]
